In [170]:
# %load corporate_credit_class_12.14.20.py
"""Copy of Corporate Credit Class 11.30.20.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1psTBioVUZ7-rSwY4uG4Nqyy2QePacCm0
"""

import sqlite3
import numpy as np
import pandas as pd
from sqlite3 import Error
import math
import blpapi
import pdblp
from datetime import date

#to-dos:
#function to pull data from excel - rather than enter manually for each name; make templates for each sector
#add prod growth, realized prices for commodities, fixed costs, capex/rev, cash, WC/rev, tax/rev to db
#add capital structure to db - individual cusips? or type of debt?

#E&P methods - leverage trajectory at different oil prices
#asset valuation - reserves, 2P, 3P, etc (think Barclays has a recovery analyses based on this )

#done to-dos:
#market pricing
#record benchmark ISINs in db 
#should probably have separate class for database functions - insert new record, make changes, pull data, etc

#------

# conn = sqlite3.connect("test.db")
# c = conn.cursor()

# c.execute("Create TABLE stocks2 (date text)")

OilPrices = [40, 50, 60, 70]
NGPrices = [2.5,2.9,3.0,3.0]
currentYear = 2020.25
currentQuarter = 2

CurrentDB = "cc_v3_test.db"
TableDict = {"E&P":"EandP","Midstream":"MIDSTREAM"}


class CorporateCredit():
  def __init__(self, name, cusip, ticker, rating, sector, country):
    self.name = name
    self.cusip = cusip
    self.ticker = ticker
    self.rating = rating
    self.sector = sector
    self.country = country

  def Print_Description(self):
    print(f"Name: ",self.name)
    print(f"Ticker: ",self.ticker)
    print(f"Rating: ",self.rating)
    print(f"Sector: ",self.sector)
    print(f"Country: ",self.country)
    
  def GetTable(self):    
    return TableDict[self.sector]
    
    
  def DB_Update(self,db,table,field,value):
    conn = sqlite3.connect(db)
    c = conn.cursor()

        #FIGURE OUT HOW to check if field exists in table

#     Query = "INSERT INTO " + table + " ("+ field + ") VALUES " + "(?)"
#     Query = "UPDATE " + table + " SET "+ field + "= " + value +" WHERE " + "NAME = " + self.name

#     c.execute(Query)
    
  
    Query = """
    UPDATE 
        {tablename} 
    SET
        {fieldname}=(?)   
    WHERE 
       name=(?)
    """.format(tablename=table,fieldname=field)
    c.execute(Query,(value,self.name))

    conn.commit()
    print("DB_Update")



  def Capital_Structure(self):
    
    Cap_Structure={}
    FirstLien=int(input("1st lien debt o/s: "))
    SecondLien=int(input("2nd lien debt o/s: "))
    Unsec=int(input("Unsecured debt o/s: "))
    Sub=int(input("Subordinated debt o/s: "))
        

    
    

  def Benchmark_Bonds(self,BenchmarkArray):
    con = pdblp.BCon(debug=False,port=8194,timeout=5000)
    con.start()
    df = con.ref(BenchmarkArray,'MATURITY')
    
    x=df.value
    currentYear = date.today().year
    self.benchmarkdict={}
    line=0

    for rows in x:    
        yearsToMaturity=rows.year-currentYear
        if (yearsToMaturity > 3 and yearsToMaturity <=6):
            self.benchmarkdict['FiveY_CUSIP']=df.ticker.iloc[line]
        elif (yearsToMaturity > 6 and yearsToMaturity <=7):
            self.benchmarkdict['SevenY_CUSIP']=df.ticker.iloc[line]
        elif (yearsToMaturity > 7 and yearsToMaturity <=15):
            self.benchmarkdict['TenY_CUSIP']=df.ticker.iloc[line]
        elif (yearsToMaturity > 15):
            self.benchmarkdict['ThirtyY_CUSIP']=df.ticker.iloc[line] 
        line+=1
        
    for key in self.benchmarkdict:
        self.DB_Update(CurrentDB, self.GetTable(),key,self.benchmarkdict[key])

        
    
    


  def Set_Targets(self):
        
    self.benchmarktargetdict={}    
    input_choice = input("Bond/stock/cds (b/s/c)?")
    if input_choice=='b':
        self.benchmarktargetdict['type'] = input("Spread or yield (s/y):")       

        maturity = input("Enter maturity (five/seven/ten/thirty):")
        while maturity not in A.benchmarkdict.keys():
            print("Maturity not in benchmark list")
            maturity = input("Enter maturity (five/seven/ten/thirty):")

        self.benchmarktargetdict[maturity] = input("Enter target: ")
    
    #link to database-input function

    
   
  def Market_Data(self,bond,field):
    con = pdblp.BCon(debug=False,port=8194,timeout=5000)
    con.start()

    # df=con.bdh(self.ticker+" equity",'px_last','20201211','20201214')
    # print(df.index)
    cusip = self.benchmarkdict[bond]
    df = con.ref(cusip,field)
    print(df.iloc[0]['value'])

    
#     df = con.ref(self.ticker+" equity",'PX_LAST')
#     df2 = con.ref("US037411BF15 corp",['PX_LAST','MATURITY'])
#     df3 = con.ref("US037411BF15 corp",'MATURITY')
#     print(df2)
#     print(df2.iloc[0]['value'])
#     print(df2.iloc[1]['value'])



  def makeDBTable (self):
    Col = " ("
    DBname = input("Enter name of DB: ")
    Tablename = input("Enter name of table you want to make: ")

    while (True):
      n = 0
      x = input("Enter column name ('q' when finished): ")
      if (x=='q'):
        break
      else:
        Col += (x+" text"+",")

      #n+=1*9--+9

    Col = Col[:-1]
    Col+=")"
    print(Col)

    conn = sqlite3.connect(DBname)
    c = conn.cursor()
    query = 'CREATE TABLE {}'.format(Tablename) + Col
    print(query)
    c.execute(query)

    #c.execute("CREATE TABLE {} ?",(Tablename,),Col)
    conn.commit()




  def makeProj (self, qtrs):

    FCF = ['Revenue','Variable Costs','Fixed Costs','EBITDA', 'Less: Capex', 'Less: Working Cap', 'Less: Taxes', 'FCF', 'Beginning Cash', 'Ending Cash']

    revArray = np.array([self.CurrentFCF['rev']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
    vcostsArray = np.array([self.CurrentFCF['var_costs']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
    fcostsArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['f_costs'],dtype=int)
    ebitdaArray = revArray+vcostsArray+fcostsArray
    capexArray = revArray * self.CurrentFCF['capex/rev']
    wcArray = revArray * self.CurrentFCF['WC/rev']
    taxArray = ebitdaArray * self.CurrentFCF['tax']  #to fix --> need interest exp
    fcfArray = ebitdaArray + capexArray + wcArray + taxArray # need interest expense

    begCashArray = np.zeros(qtrs+1,dtype=int)
    begCashArray[0] = self.CurrentFCF['cash']
    endCashArray = np.zeros(qtrs+1,dtype=int)
    endCashArray[0] = fcfArray[0]+begCashArray[0]


    for i in range(1,qtrs+1):
      begCashArray[i] = endCashArray[i-1]
      endCashArray[i] = begCashArray[i] + fcfArray[i]


    FCF_statement=[revArray,vcostsArray,fcostsArray,ebitdaArray,capexArray,wcArray,taxArray,fcfArray,begCashArray,endCashArray]
    FCF_df=pd.DataFrame(FCF_statement,columns = self.makeDateColumn(qtrs), index = FCF, dtype=int)

      # print(ebitdaArray)
      # print(begCashArray)
      # print(endCashArray)


    return FCF_df

  def makeDateColumn(self,qtrs):
    yearColumns = ['Q'+str(currentQuarter)+'/'+str(math.floor(currentYear))]
    for x in range (qtrs):
      yearColumns.append('Q'+str(((currentQuarter+x) % 4)+1)+'/'+str(math.floor(currentYear+(x+1)*0.25)))
    return yearColumns



class EnergyCredit(CorporateCredit):
  def __init__(self, name, cusip, ticker, rating, sector, country):
    super().__init__(name, cusip, ticker, rating, sector, country)






class EPCredit(EnergyCredit):
  def __init__(self, name, cusip, ticker, rating, sector, country):
    super().__init__(name, cusip, ticker, rating, sector, country)
     # think about what parameters fully define an individual E&P Credit - Cash cost, production for each commodity, realized px, etc

  def DB_Input(self):
    DebtEBITDA = float(input("Enter Debt/EBITDA: "))
    CashCost = float(input("Enter cash cost: "))
    OilProd = float(input("Enter Daily Oil Prod (bbd/d): "))
    NGProd = float(input("Enter Daily Natural Gas Prod (mmcf/d):  "))
    NGLProd = float(input("Enter Daily NGL Prod (bbd/d):  "))

    conn = sqlite3.connect("cc_v1.db")
    c = conn.cursor()
    c.execute("INSERT INTO EandP VALUES (?,?,?,?,?,?)",(self.name, CashCost, OilProd, NGLProd, NGProd, DebtEBITDA))
    conn.commit()
    # c.execute("SELECT * FROM EandP")
    # print(c.fetchone())
            
  
        



  def SetHedges(self):
    print(OilPrices[0])
    n = int(input("How many years are hedged: "))
    Hedge = [[0 for i in range(n)] for j in range(7)]
    #[year 0, year 1, ...]
    #[oil volume]
    #[oil realized price]
    #[NGL volume]
    #[NGL realized price]
    #[NG volume]
    #[NG realized price]

    #print(type(n))
    for i in range(0,n):
      print("Year "+str(i+1)+" :")
      TypeHedge = input("Oil hedged through (f)utures or (o)ptions?")

      if TypeHedge=='f':
        Hedge[1][i] = int(input("Oil volume hedged: "))
        Hedge[2][i] = int(input("Hedged Price: "))

      elif TypeHedge=='o':
        Hedge[1,i] = int(input("Oil volume hedged: "))
        Floor = int(input("Put Strike: "))
        Ceiling = int(input("Written Call Strike: "))
        SoldPut = int(input("Written Put Strike: "))
        Hedge[2][i] = OilPrices[i] - max(OilPrices[i]-Ceiling,0)+max(Floor-OilPrices[i],0)-max(SoldPut-OilPrices[i],0)

      TypeHedge = input("Gas hedged through (f)utures or (o)ptions?")

      if TypeHedge=='f':
        Hedge[5][i] = int(input("Gas volume hedged: "))
        Hedge[6][i] = int(input("Hedged Price: "))

      elif TypeHedge=='o':
        Hedge[5][i] = int(input("Gas volume hedged: "))
        Floor = int(input("Put Strike: "))
        Ceiling = int(input("Written Call Strike: "))
        SoldPut = int(input("Written Put Strike: "))
        Hedge[6][i] = OilPrices[i] - max(OilPrices[i]-Ceiling,0)+max(Floor-OilPrices[i],0)-max(SoldPut-OilPrices[i],0)

      TypeHedge = input("NGL hedged through (f)utures or (o)ptions?")

      if TypeHedge=='f':
        Hedge[3][i] = int(input("NGL volume hedged: "))
        Hedge[4][i] = int(input("Hedged Price: "))

      elif TypeHedge=='o':
        Hedge[3][i] = int(input("NGL volume hedged: "))
        Floor = int(input("Put Strike: "))
        Ceiling = int(input("Written Call Strike: "))
        SoldPut = int(input("Written Put Strike: "))
        Hedge[4][i] = OilPrices[n] - max(OilPrices[i]-Ceiling,0)+max(Floor-OilPrices[i],0)-max(SoldPut-OilPrices[i],0)

    return(Hedge)

  def setCurrentFCF(self):
    conn = sqlite3.connect("cc_v1_test.db")
    c = conn.cursor()
    c.execute("SELECT OilperDay from EandP WHERE name=?",(self.name,))
    #print(c.fetchone()[0])
    #c.fetchone()[0]
    #print(c.fetchone()[0])
    OilProd = int(c.fetchone()[0])
    c.execute("SELECT NGLperDay from EandP WHERE name=?",(self.name,))
    NGLProd = int(c.fetchone()[0])
    c.execute("SELECT NGperDay from EandP WHERE name=?",(self.name,))
    NGProd = int(c.fetchone()[0])
    c.execute("SELECT CashCost from EandP WHERE name=?",(self.name,))
    CashCost = int(c.fetchone()[0])

    TotalProd = OilProd + NGLProd + NGProd/6
    Oil_rev = OilProd * OilPrices[0] * 0.365/4
    NGL_rev = NGLProd * OilPrices[0] * 0.365/4
    NG_rev = NGProd * NGPrices[0] * 0.365/4

    Rev = Oil_rev + NGL_rev + NG_rev
    Prod_g = 0.01
    Rev_g = Prod_g
    Var_costs = -(CashCost * TotalProd * 0.365/4)
    F_costs = -70
    Cash = 162
    Capex_rev = -0.27
    WC_rev = -0.06
    Taxes = -0.11 #% of EBITDA

    self.CurrentFCF = {'rev': Rev, 'rev_g': Rev_g, 'var_costs': Var_costs, 'f_costs': F_costs, 'cash': Cash, 'capex/rev': Capex_rev, 'WC/rev': WC_rev, 'tax':Taxes}











A = EPCredit("Apache","xx","APA",["BBB-","BBB"],"E&P","USA")

A.Print_Description()
# A.Market_Data()
# A.Benchmark_Bonds("US037411BH70")
A.Benchmark_Bonds(["US037411BH70 corp","US037411BE40 corp","US037411BF15 corp","US037411BA28 corp"])
print(A.benchmarkdict['FiveY_CUSIP'])
# A.Market_Data('five','px_last')
print(A.benchmarkdict)
# A.Set_Targets()




# def Benchmark_Bonds(self,FiveYr,SevenYr,TenYr,ThirtyYr):
    
    
# #A.DB_Input()
# # A.setCurrentFCF()
# # print(A.CurrentFCF)
# # fcf = A.makeProj(4)
# # fcf
# Hedge = A.SetHedges()
# print(Hedge)
# A.makeDBTable()



Name:  Apache
Ticker:  APA
Rating:  ['BBB-', 'BBB']
Sector:  E&P
Country:  USA
DB_Update
DB_Update
DB_Update
US037411BH70 corp
{'FiveY_CUSIP': 'US037411BH70 corp', 'TenY_CUSIP': 'US037411BF15 corp', 'ThirtyY_CUSIP': 'US037411BA28 corp'}
